## 红黑树

### 红黑树节点

In [2]:
class RB_Node:
    '''0 is black, 1 is red'''
    def __init__(self, val, color = 1):
        self.val = val
        self.color = color
        self.left = None
        self.right = None
        self.parent = None
    
    def is_black_node(self):
        return self.color == 0

    def set_black_node(self):
        self.color = 0
    
    def set_red_node(self):
        self.color = 1

    def print(self):
        '''中序遍历打印树'''
        if self.left: self.left.print()
        # print((self.val, self.color))
        print(self.val)
        if self.right: self.right.print()

### 左旋与右旋

In [3]:
class RB_Tree:
    def __init__(self, root=None):
        self.root = root
    
    def print(self):
        self.root.print()

    def left_rotate(self, node: RB_Node):
        '''
        左旋操作，分为三步：
        1. 将 r.parent 置为 p（可能会更新 root），根据 node 的位置把 r 连接至 p.left 或 p.right
        2. 将 node.parent 置为 r，r.left 置为 node
        3. 将 node.right 置为 rl，若 rl 存在，将 rl.parent 置为 node，
        * 左旋示意图：对节点x进行左旋
        *     p                       p
        *    /                       /
        *   node                    r
        *  / \                     / \
        * x   r     ----->      node  rr
        *    / \                /  \
        *   rl  rr             x   rl
        '''

        p = node.parent 
        r = node.right
        rl = r.left

        # 1 r 连到父节点，若父节点为空（node 为 root），更新 root
        r.parent = p
        if not p: self.root = r 
        else:
            # 1.1 根据 node 相对于 p 的位置来连接 p 与 r
            if (p.left == node): p.left = r 
            else: p.right = r 

        # 2. 连接 node 与 r
        node.parent = r
        r.left = node 
        
        # 3. 连接 rl 与 node
        node.right = rl 
        if (rl): rl.parent = node 

    def right_rotate(self, node: RB_Node):
        ''' 
        右旋操作，分为三步：
        1. 将 l.parent 置为 p（可能会更新 root），根据 node 的位置把 l 连接至 p.left 或 p.right
        2. 将 node.parent 置为 l，l.right 置为 node
        3. 将 node.left 置为 lr，若 lr 存在，将 lr.parent 置为 node，
        * 右旋示意图：对节点y进行右旋
        *         p                   p
        *        /                   /
        *      node                 l
        *     /    \               / \
        *    l      r   ----->   ll  node
        *   / \                      /  \
        * ll   lr                   lr   r
        '''
        p = node.parent
        l = node.left
        lr = l.right

        l.parent = p
        if not p: self.root = l
        else:
            if (p.left == node): p.left = l 
            else: p.right = l 

        node.parent = l 
        l.right = node 

        node.left = lr
        if (lr): lr.parent = node 

#### 测试

In [14]:
from collections import deque
from typing import List

def list2Tree(l: List[int]) -> RB_Node:
    assert (len(l) > 0), "len(l) must > 0"
    tree = RB_Tree(RB_Node(l[0], 0))
    deq = deque()
    curp = tree.root
    for i in range(1, len(l)):
        curs = RB_Node(l[i])
        if (curp.right): curp = deq.popleft()
        if (curp.left): curp.right = curs
        else: curp.left = curs
        curs.parent = curp 
        deq.append(curs)
    return tree 

In [7]:
def printTree(root):
    def maxDepth(root):
        if not root:
            return -1
        leftDepth = maxDepth(root.left)
        rightDepth = maxDepth(root.right)
        return max(leftDepth, rightDepth) + 1
    h = maxDepth(root)

    m = h + 1
    n = pow(2, m) - 1
    res = [["" for i in range(n)] for i in range(m)]
    nodes = [(root, 0, (n-1)//2)]
    
    while nodes:
        cur, r, c = nodes.pop()
        res[r][c] = str(cur.val)
        if cur.left:
            nodes.append((cur.left, r+1, c-pow(2,h-r-1)))
        if cur.right:
            nodes.append((cur.right, r+1, c+pow(2,h-r-1)))
    
    return res

In [8]:
t = list2Tree([1,2,3,4,5,6])

In [9]:
t.right_rotate(t.root)

In [10]:
printTree(t.root)

[['', '', '', '', '', '', '', '2', '', '', '', '', '', '', ''],
 ['', '', '', '4', '', '', '', '', '', '', '', '1', '', '', ''],
 ['', '', '', '', '', '', '', '', '', '5', '', '', '', '3', ''],
 ['', '', '', '', '', '', '', '', '', '', '', '', '6', '', '']]

### 插入

在讨论插入情况之前，先做如下表述上的约定：新插入节点为 Node，其父节点为 P， 祖父节点为 G，叔叔节点为 U，插入节点与父节点的相对位置为 L/R。

新插入节点初始化为红色节点，插入节点时有以下几种情况：

1. 插入节点为根节点，直接将节点颜色置为黑色
2. 插入节点的父节点为黑色，不需要做任何操作
3. 插入节点的父节点为红色，需要进行调整
    1. 插入节点的叔叔节点为红色，将父节点和叔叔节点置为黑色，祖父节点置为红色，将祖父节点作为新的插入节点，继续调整
    2. 插入节点的叔叔节点为黑色，且插入节点与父节点同侧（LL/RR），以祖父节点为支点进行反向旋转（R/L），将父节点置为黑色，祖父节点置为红色。
    3. 插入节点的叔叔节点为黑色，且插入节点与父节点异侧（LR/RL），以父节点为支点进行反向旋转（L/R），转化为 3.2。

In [35]:
class RB_Tree:
    def __init__(self, root=None):
        self.root = root
    
    def print(self):
        self.root.print()

    def left_rotate(self, node: RB_Node):
        p = node.parent 
        r = node.right
        rl = r.left

        # 1. r 连到父节点，若父节点为空（node 为 root），更新 root
        r.parent = p
        if not p: self.root = r 
        else:
            # 1.1 根据 node 相对于 p 的位置来连接 p 与 r
            if (p.left == node): p.left = r 
            else: p.right = r 

        # 2. 连接 node 与 r
        node.parent = r
        r.left = node 
        
        # 3. 连接 rl 与 node
        node.right = rl 
        if (rl): rl.parent = node 

    def right_rotate(self, node: RB_Node):
        p = node.parent
        l = node.left
        lr = l.right

        l.parent = p
        if not p: self.root = l
        else:
            if (p.left == node): p.left = l 
            else: p.right = l 

        node.parent = l 
        l.right = node 

        node.left = lr
        if (lr): lr.parent = node  

    def insert_rebalance(self, node):
        # 目前正在处理的节点
        cur = node  
        # 父节点为黑色不用处理，2
        while (cur.parent.color == 1):  
            p = cur.parent
            g = cur.parent.parent
            # L
            if (p == g.left): 
                u = g.right
                # uncle 红色，3.1
                if (u.color == 1):  
                    p.color = u.color = 0
                    g.color = 1
                    # 更新当前处理节点切换为祖父节点
                    cur = g  
                else:
                    # LR, 3.3 转化为 3.2
                    if (cur == p.right): self.left_rotate(p)
                    p.color = 0
                    g.color = 1
                    self.right_rotate(g)
            # R
            else:
                u = g.left
                if (u.color == 1):
                    p.color = u.color = 0
                    g.color = 1
                    cur = g
                else:
                    # RL, 转为 3.2
                    if (cur == p.left): self.right_rotate(p)
                    self.left_rotate(g)
                    p.color = 0
                    g.color = 1
            
            if (cur == self.root): break
        # 根节点始终为黑色
        self.root.color = 0

In [36]:
def printTree(root, color = True):
    def maxDepth(root):
        if not root: return -1
        leftDepth = maxDepth(root.left)
        rightDepth = maxDepth(root.right)
        return max(leftDepth, rightDepth) + 1
    
    h = maxDepth(root)
    m = h + 1
    n = pow(2, m) - 1
    res = [["" for i in range(n)] for i in range(m)]
    nodes = [(root, 0, (n-1)//2)]
    
    while nodes:
        cur, r, c = nodes.pop()
        res[r][c] = str(cur.val)
        if color: res[r][c] += " " + str(cur.color)
        if cur.left: nodes.append((cur.left, r+1, c-pow(2,h-r-1)))
        if cur.right: nodes.append((cur.right, r+1, c+pow(2,h-r-1)))
    
    return res

In [37]:
t = list2Tree([1, 2, 3, 4])

In [38]:
printTree(t.root)

[['', '', '', '1 0', '', '', ''],
 ['', '2 1', '', '', '', '3 1', ''],
 ['4 1', '', '', '', '', '', '']]

In [39]:
t.insert_rebalance(t.root.left.left)

In [40]:
printTree(t.root)

[['', '', '', '1 0', '', '', ''],
 ['', '2 0', '', '', '', '3 0', ''],
 ['4 1', '', '', '', '', '', '']]

## 节点删除及树修复

红黑树的节点删除操作逻辑上分为两部分，节点的删除以及树的重新平衡，实际上在许多代码中也将删除操作分为两个函数来完成，这里的实现也按照此逻辑。

笔者认为，网络上包括书籍中的很多关于红黑树删除的教程，之所以让人看了云里雾里，是因为它们在演示中并没有将删除以及重新平衡这两个步骤分开来展示，在列举各种情况时总是把删除以及重新平衡的情况放一起说完，这样就会导致逻辑的混乱，且不利于记忆写代码的步骤。笔者试图按照自己看了许多相关博客后的理解，力求把删除这件事情说明白。

### 节点删除

就节点删除这一个步骤来看，红黑树的节点删除操作与BST（二叉搜索树）基本相同，可分为三种情况：


1. 删除只有一个子树的节点：删除原节点，用单独的子节点代替被删除节点
2. 删除没有子树的节点：直接删除原节点，相当于用一个空节点代替了被删除节点
3. 删除有两个子树的节点：寻找后继或前继节点（这里选择后继），将待删除节点的值变为后继节点的值，再删除后继节点，删除后继节点一定会变成 1 或 2 的情况（可以思考一下为什么）

但由于红黑树的特殊性（多了一个颜色属性），因此要记录被删除节点的颜色，以应用到后续的平衡步骤中。

In [42]:
def findMin(self, node):
    """
    找到以 node 节点为根节点的树的最小值节点 并返回
    """
    temp_node = node
    while temp_node.left: temp_node = temp_node.left
    return temp_node

def transplant(self, tree, node1, node2):
    """
    辅助函数，用 node2 替换 node1
    """
    p1 = node1.parent
    if not p1: tree.root = node2
    elif (node1 == p1.left): p1.left = node2
    elif (node1 == p1.right): p1.right = node2
    if node2: node2.parent = p1

用 node2 替换 node1，这里只考虑了 node1 的parent 指针与 node2 的交接而没有考虑两个节点的 left 及 right 指针的问题，因为在大部分操作中，node2 为 node1 唯一的子节点，而 node1 即将被删除，不需考虑 left 及 right 指针，需要维护 left 及 right 指针时会在执行完函数后单独处理。

In [44]:
def delete_node(self, node):
    node_color = node.color
    temp_node = None  # 替换到被删除节点（node）处的节点
    
    # ===================== case1 与 case2 ================== //
    # node 本身被删除
    if not node.left:
        temp_node = node.right
        self.transplant(node, node.right)
    elif not node.right:
        temp_node = node.left
        self.transplant(node, node.left)
    # ========================= case 3 ====================== //
    # node 的后继被删除
    else:
        # 找到后继节点，及右子树中的最小节点，该节点将代替 node 被删除
        successor = self.findMin(node.right)
        node_color = successor.color
        # temp_node 将在 successor 被删除后代替它
        temp_node = successor.right
        # 将 node 替换为 successor，相当于仅替换值
        node.val = successor.val
        # 删除 successor，用 temp_node 替代
        self.transplant(successor, temp_node)
        
    # 这里的 delete_rebalance 为红黑树的重新平衡操作，需要注意的是，
    # 重新平衡的操作总是在被删除节点的位置进行的，即上面维护的 temp_node 最终的位置
    # 仅在被删除节点的颜色（node_color 维护）为黑色时才需重新平衡树
    if (node_color == 0): self.delete_rebalance(temp_node)

### 红黑树修复

在讨论红黑树的重新平衡之前，回顾一下在红黑树的节点删除部分我们做了什么：首先，根据将要被删除的节点的孩子数量，来决定删除策略，其中零或一个孩子的情况较好处理，而两个孩子的情况则可以通过寻找后继节点的方式转换为前两种情况，需要注意的是，前两种情况删除的 node 位置即为最初的 node 位置，而后一种情况删除的节点位置已经不在最初的 node 的位置了，而是其后继所在的位置。

在删除完节点后，temp_node 所指向的节点为 **替代被删除节点的节点**，它的位置也在被删除节点的位置，后续的所有的修复情况均以 temp_node 的位置及其相关节点的颜色作为判断标准。

弄清楚了这些，才能是我们更加容易理解红黑树修复的各种情况。

这里以 N 代表 temp_node，S 代表 N 的兄弟节点，P 代表 N 与 S 的父节点，SL 与 SR 分别代表 S 的左右节点，红黑树修复有以下六种情况：

1. N 为新的根节点，这说明原始的

In [45]:
def delete_rebalance(self, node):
    while (node != self.root and node.color == 0):
        if (node == node.parent.left):
            s = node.parent.right
            # case2：兄弟节点为红色节点，旋转父节点
            if (s.color == 1):
                node.parent.color = 1
                s.color = 0
                self.left_rotate(node.parent)
                s = node.parent.right  # 更新兄弟节点
            # case3: 兄弟节点的两个子节点均为黑色，父节点也为黑色？
            if (not s.left or s.left.color == 0) and (not s.right or s.right.color == 0):
                s.color = 1
                node = node.parent  # 向上继续处理
            else:
                # case5: 兄弟节点同侧的节点为黑色
                if (not s.right or s.right.color == 0):
                    s.color = 1
                    s.left.color = 0
                    self.right_rotate(s)
                    s = node.parent.right  # 更新兄弟节点
                # 处理之后变为 case6
                # case6: 兄弟节点的同侧节点为红色
                s.color = node.parent.color
                node.parent.color = 0
                s.right.color = 0
                self.left_rotate(node.parent)
            node = self.root 
            break
        # 对称操作
        else:
            s = node.parent.left
            if (s.color == 1):
                node.parent.color = 1
                s.color = 0
                self.right_rotate(node.parent)
                s = node.parent.left
            if (not s.left or s.left.color == 0) and (not s.right or s.right.color == 0):
                s.color = 1
                node = node.parent
            else:
                if (not s.left or s.left.color == 0):
                    s.color = 1
                    s.right.color = 0
                    self.left_rotate(s)
                    s = node.parent.left
                s.color = node.parent.color
                node.parent.color = 0
                s.left.color = 0
                self.right_rotate(node.parent)
            node = self.root
            break
        node.color = 0